In [1]:
import init

import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
from scipy.stats import entropy

from webspot.graph.graph_loader import GraphLoader

In [2]:
loader = GraphLoader(
    json_path='/Users/marvzhang/projects/tikazyq/webspot/data/quotes.toscrape.com/json/http___quotes_toscrape_com_.json',
    embed_walk_length=5,
)

loader.run()

/Users/marvzhang/projects/tikazyq/webspot/venv/lib/python3.10/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
/Users/marvzhang/projects/tikazyq/webspot/venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [3]:
loader.node_ids_enc.transform(loader.nodes_ids_tensor)

ValueError: y contains previously unseen labels: [0, 1, 2, 3, 4, 5]

In [4]:
loader.nodes_embedded_tensor

tensor([[  0,   1,   9, 116, 130, 131],
        [  1,   2,   6,   7,   8,  -1],
        [  2,   6,   7,   8,  -1,  -1],
        [  3,   4,   5,  -1,  -1,  -1],
        [  4,   5,  -1,  -1,  -1,  -1],
        [  5,  -1,  -1,  -1,  -1,  -1],
        [  6,   7,   8,  -1,  -1,  -1],
        [  7,   8,  -1,  -1,  -1,  -1],
        [  8,  -1,  -1,  -1,  -1,  -1],
        [  9, 116, 128, 129,  -1,  -1],
        [ 10,  43,  45,  47,  -1,  -1],
        [ 11,  12,  -1,  -1,  -1,  -1],
        [ 12,  -1,  -1,  -1,  -1,  -1],
        [ 13,  14,  -1,  -1,  -1,  -1],
        [ 14,  -1,  -1,  -1,  -1,  -1],
        [ 15,  -1,  -1,  -1,  -1,  -1],
        [ 16,  19,  -1,  -1,  -1,  -1],
        [ 17,  -1,  -1,  -1,  -1,  -1],
        [ 18,  -1,  -1,  -1,  -1,  -1],
        [ 19,  -1,  -1,  -1,  -1,  -1],
        [ 20,  -1,  -1,  -1,  -1,  -1],
        [ 21,  -1,  -1,  -1,  -1,  -1],
        [ 22,  23,  -1,  -1,  -1,  -1],
        [ 23,  -1,  -1,  -1,  -1,  -1],
        [ 24,  25,  -1,  -1,  -1,  -1],


In [6]:
idx = loader.nodes_embedded_tensor[10]
idx

tensor([10, 43, 45, 47, -1, -1])

In [7]:
loader.nodes_features_tensor[idx]

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0

In [8]:
loader.nodes_features_tensor[idx].shape

torch.Size([6, 106])

In [9]:
loader.nodes_embedded_tensor

tensor([[  0,   1,   9, 116, 130, 131],
        [  1,   2,   6,   7,   8,  -1],
        [  2,   6,   7,   8,  -1,  -1],
        [  3,   4,   5,  -1,  -1,  -1],
        [  4,   5,  -1,  -1,  -1,  -1],
        [  5,  -1,  -1,  -1,  -1,  -1],
        [  6,   7,   8,  -1,  -1,  -1],
        [  7,   8,  -1,  -1,  -1,  -1],
        [  8,  -1,  -1,  -1,  -1,  -1],
        [  9, 116, 128, 129,  -1,  -1],
        [ 10,  43,  45,  47,  -1,  -1],
        [ 11,  12,  -1,  -1,  -1,  -1],
        [ 12,  -1,  -1,  -1,  -1,  -1],
        [ 13,  14,  -1,  -1,  -1,  -1],
        [ 14,  -1,  -1,  -1,  -1,  -1],
        [ 15,  -1,  -1,  -1,  -1,  -1],
        [ 16,  19,  -1,  -1,  -1,  -1],
        [ 17,  -1,  -1,  -1,  -1,  -1],
        [ 18,  -1,  -1,  -1,  -1,  -1],
        [ 19,  -1,  -1,  -1,  -1,  -1],
        [ 20,  -1,  -1,  -1,  -1,  -1],
        [ 21,  -1,  -1,  -1,  -1,  -1],
        [ 22,  23,  -1,  -1,  -1,  -1],
        [ 23,  -1,  -1,  -1,  -1,  -1],
        [ 24,  25,  -1,  -1,  -1,  -1],


In [12]:
loader.nodes_features_tensor[loader.nodes_embedded_tensor].shape

torch.Size([145, 6, 106])

In [13]:
loader.nodes_features_tensor[loader.nodes_embedded_tensor]

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0

## Nodes Features (tags + attributes)

In [14]:
feat_tags_attrs = loader.nodes_features_tensor
feat_tags_attrs

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)

In [15]:
X_feat_tags_attrs = feat_tags_attrs.detach().numpy()
X_feat_tags_attrs

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
X_feat_tags_attrs = normalize(X_feat_tags_attrs, norm='l1', axis=1)
X_feat_tags_attrs

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0.5, 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]], dtype=float32)

## Nodes Features (node2vec)

In [103]:
feat_node2vec = loader.nodes_features_tensor[loader.nodes_embedded_tensor].sum(dim=1)
feat_node2vec.shape

torch.Size([122, 148])

In [104]:
X_feat_node2vec = feat_node2vec.detach().numpy()
X_feat_node2vec.shape

(122, 148)

In [105]:
X_feat_node2vec = normalize(X_feat_node2vec, norm='l1', axis=1)
X_feat_node2vec

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Nodes Features (all: tags + attributes + node2vec)

In [106]:
X = np.concatenate((X_feat_tags_attrs, X_feat_node2vec), axis=1)
X.shape

(122, 296)

In [107]:
X = normalize(X, norm='l2', axis=1)
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## DBSCAN Clustering

In [108]:
dbscan = DBSCAN()
dbscan.fit(X)

DBSCAN()

In [109]:
labels = pd.Series(dbscan.labels_)
labels.value_counts()

-1    86
 4    10
 2     8
 3     7
 1     6
 0     5
dtype: int64

In [110]:
labels_counts = labels.value_counts()
labels_counts[labels_counts.index > -1]

4    10
2     8
3     7
1     6
0     5
dtype: int64

In [111]:
for i in labels_counts[labels_counts.index > -1].index.tolist():
    top_label = labels_counts.index.values[i]
    top_label

    top_nodes_idx = labels[labels == top_label].index.values
    top_nodes_idx

    nodes = np.array(loader.nodes)[top_nodes_idx]
    nodes = sorted(nodes, key=lambda n: n.graph_node_id)
#     nodes = list(map(lambda n: f'[{loader.graphs_id_file_map.get(n.graph_id)}] {n.graph_node_id} -> {n.graph_parent_id}: {n.features}', nodes))
    nodes = list(map(lambda n: f'{n.graph_node_id} -> {n.graph_parent_id}: {n.features}', nodes))
    print(np.array(nodes[:50]))
    print('\n\n')

["0.123 -> 0.122: [('tag', 'p')]" "0.125 -> 0.122: [('tag', 'p')]"
 "0.129 -> 0.122: [('tag', 'p')]" "0.46 -> 0.45: [('tag', 'p')]"
 "0.49 -> 0.48: [('tag', 'p')]" "0.53 -> 0.52: [('tag', 'p')]"]



["0.61 -> 0.60: [('tag', 'li'), ('class', 'module-list-item')]"
 "0.64 -> 0.60: [('tag', 'li'), ('class', 'module-list-item')]"
 "0.67 -> 0.60: [('tag', 'li'), ('class', 'module-list-item')]"
 "0.70 -> 0.60: [('tag', 'li'), ('class', 'module-list-item')]"
 "0.73 -> 0.60: [('tag', 'li'), ('class', 'module-list-item')]"
 "0.76 -> 0.60: [('tag', 'li'), ('class', 'module-list-item')]"
 "0.79 -> 0.60: [('tag', 'li'), ('class', 'module-list-item')]"
 "0.82 -> 0.60: [('tag', 'li'), ('class', 'module-list-item')]"]



["0.62 -> 0.61: [('tag', 'span')]" "0.65 -> 0.64: [('tag', 'span')]"
 "0.68 -> 0.67: [('tag', 'span')]" "0.71 -> 0.70: [('tag', 'span')]"
 "0.74 -> 0.73: [('tag', 'span')]" "0.77 -> 0.76: [('tag', 'span')]"
 "0.80 -> 0.79: [('tag', 'span')]"]



["0.101 -> 0.91: [('tag', 'li'), ('clas

## Filter Clusters

In [112]:
df_nodes = pd.DataFrame({
    'id': [n.id for n in loader.nodes],
    'parent_id': [n.parent_id for n in loader.nodes],
    'label': dbscan.labels_,
})
df_nodes

,id,parent_id,label
0,16,0,-1
1,17,16,0
2,18,16,-1
3,19,18,-1
4,20,19,-1
...,...,...,...
117,133,19,0
118,134,19,0
119,135,19,-1
120,136,19,0


In [113]:
df_nodes = df_nodes[df_nodes.label != -1]
df_nodes.head()

,id,parent_id,label
1,17,16,0
30,46,45,1
33,49,48,1
37,53,52,1
45,61,60,2


In [114]:
df_labels = df_nodes \
    .groupby('label')[['parent_id']] \
    .agg(lambda s: entropy(s.value_counts())) \
    .rename(columns={'parent_id': 'entropy'}) \
    .sort_values(by='entropy', ascending=True) \
    .reset_index()
df_labels

,label,entropy
0,2,0.000000
1,4,0.500402
2,0,0.673012
3,1,1.242453
4,3,1.945910


In [115]:
for label in df_labels[df_labels.entropy < 0.5].label.tolist():
    # mask
    mask = df_nodes.label == label

    # entropy
    arr = df_nodes[mask].parent_id
    prob = arr.value_counts(normalize=True)
    ent = df_labels[df_labels.label == label]

    # nodes
    nodes_ids = df_nodes[mask].id.values
    nodes = np.array([n for n in loader.nodes if n.id in nodes_ids])

    # results
    print(f'label: {label}')
    print(f'entropy: {ent}')
    print(df_nodes[mask].parent_id.values)
    print(nodes)
    print('\n')

label: 2
entropy:    label  entropy
0      2      0.0
[60 60 60 60 60 60 60 60]
[{'id': 61, 'parent_id': 60, 'graph_id': 0, 'features': [('tag', 'li'), ('class', 'module-list-item')]}
 {'id': 64, 'parent_id': 60, 'graph_id': 0, 'features': [('tag', 'li'), ('class', 'module-list-item')]}
 {'id': 67, 'parent_id': 60, 'graph_id': 0, 'features': [('tag', 'li'), ('class', 'module-list-item')]}
 {'id': 70, 'parent_id': 60, 'graph_id': 0, 'features': [('tag', 'li'), ('class', 'module-list-item')]}
 {'id': 73, 'parent_id': 60, 'graph_id': 0, 'features': [('tag', 'li'), ('class', 'module-list-item')]}
 {'id': 76, 'parent_id': 60, 'graph_id': 0, 'features': [('tag', 'li'), ('class', 'module-list-item')]}
 {'id': 79, 'parent_id': 60, 'graph_id': 0, 'features': [('tag', 'li'), ('class', 'module-list-item')]}
 {'id': 82, 'parent_id': 60, 'graph_id': 0, 'features': [('tag', 'li'), ('class', 'module-list-item')]}]


